# Minimal three node network

Here, we are going to create a network with three nodes, three lines and one generator. We then solve the non-linear power flow using a Newton-Raphson.  

In [1]:
import pypsa
import numpy as np

In [2]:
network = pypsa.Network()

Add three buses

In [3]:
n_buses = 3

for i in range(n_buses):
    network.add("Bus", "My bus {}".format(i), v_nom=20.0)

network.buses

attribute,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,
My bus 0,20.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,
My bus 1,20.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,
My bus 2,20.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,


Add three lines in a ring

In [4]:
for i in range(n_buses):
    network.add(
        "Line",
        "My line {}".format(i),
        bus0="My bus {}".format(i),
        bus1="My bus {}".format((i + 1) % n_buses),
        x=0.1,
        r=0.01,
    )

network.lines

attribute,bus0,bus1,type,x,r,g,b,s_nom,s_nom_extendable,s_nom_min,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,
My line 0,My bus 0,My bus 1,,0.1,0.01,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
My line 1,My bus 1,My bus 2,,0.1,0.01,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
My line 2,My bus 2,My bus 0,,0.1,0.01,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Add a generator at bus 0

In [5]:
network.add("Generator", "My gen", bus="My bus 0", p_set=100, control="PQ")

network.generators

attribute,bus,control,type,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
My gen,My bus 0,PQ,,0.0,False,0.0,inf,0.0,1.0,100.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


In [6]:
network.generators.p_set

Generator
My gen    100.0
Name: p_set, dtype: float64

Add a load at bus 1

In [7]:
network.add("Load", "My load", bus="My bus 1", p_set=100)

network.loads

attribute,bus,carrier,type,p_set,q_set,sign
Load,,,,,,
My load,My bus 1,,,100.0,0.0,-1.0


In [8]:
network.loads.p_set

Load
My load    100.0
Name: p_set, dtype: float64

Fix the reactive power of the load

In [9]:
network.loads.q_set = 100.0

Do a Newton-Raphson power flow

In [10]:
from qalcore.dwave.qubols.encodings import RealQbitEncoding, RealUnitQbitEncoding, EfficientEncoding
quantum_solver='qubo'
quantum_solver_options = {'num_reads':100, 'num_qbits':21, 'encoding':EfficientEncoding}
# network.pf()
network.pf(x_tol=1E-6, quantum_solver=quantum_solver, quantum_solver_options=quantum_solver_options)

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object', name='snapshot')


  (0, 1)	-3960.39603960396
  (0, 0)	7920.79207920792
  (0, 2)	792.0792079207921
  (0, 3)	-396.03960396039605
  (1, 1)	7920.79207920792
  (1, 0)	-3960.39603960396
  (1, 2)	-396.03960396039605
  (1, 3)	792.0792079207921
  (2, 1)	396.03960396039605
  (2, 0)	-792.0792079207921
  (2, 2)	7920.79207920792
  (2, 3)	-3960.39603960396
  (3, 1)	-792.0792079207921
  (3, 0)	396.03960396039605
  (3, 2)	-3960.39603960396
  (3, 3)	7920.79207920792
[100.   0. 100.   0.]
[0.01591467 0.00863168 0.01820746 0.00903629]
  (0, 1)	-3850.243192815965
  (0, 0)	7731.850282621056
  (0, 3)	-417.1364958104999
  (0, 2)	675.2666844182515
  (1, 1)	7776.930612918137
  (1, 0)	-3855.8556302332827
  (1, 2)	-363.86786212649656
  (1, 3)	779.133847278236
  (2, 1)	413.3671282390518
  (2, 0)	-864.0247974141528
  (2, 3)	-3885.3523750458494
  (2, 2)	7677.910875621287
  (3, 1)	-783.5645650388358
  (3, 0)	357.242754065873
  (3, 2)	-3927.362917437067
  (3, 3)	7850.588914601586
[-0.52650069 -5.73559972  3.13268458  1.35904026]
[-0.0

INFO:pypsa.pf:Newton-Raphson solved in 5 iterations with error of 0.000001 in 26.349285 seconds


  (0, 1)	-3846.206653714712
  (0, 0)	7725.746649660733
  (0, 3)	-418.5676403155309
  (0, 2)	675.2708270316995
  (1, 1)	7772.1152738445035
  (1, 0)	-3852.147250656475
  (1, 2)	-362.0212497403407
  (1, 3)	784.6102603735104
  (2, 1)	414.6206799273752
  (2, 0)	-862.5746008121245
  (2, 3)	-3882.8204215314354
  (2, 2)	7669.953200989542
  (3, 1)	-777.2114185229153
  (3, 0)	355.2147105097436
  (3, 2)	-3925.961173919968
  (3, 3)	7846.101504756735
[ 6.39569191e-05  1.07696616e-04 -1.97029766e-06 -1.16491940e-05]
[2.02772441e-08 2.39967386e-08 9.59869542e-10 4.79934771e-10]


{'n_iter': SubNetwork  0
 snapshot     
 now         5,
 'error': SubNetwork             0
 snapshot                
 now         8.508025e-07,
 'converged': SubNetwork     0
 snapshot        
 now         True}

In [11]:
from sympy.matrices import Matrix, SparseMatrix

In [12]:
import scipy.sparse as spsp 

In [13]:
A = np.random.rand(4,4)
A += A.T 

Asp = spsp.csc_matrix(A)
print(Asp)

SparseMatrix(4,4, dict(Asp.todok().items()))

  (0, 0)	1.220157568493924
  (1, 0)	1.2001028347032583
  (2, 0)	1.0981924683541506
  (3, 0)	1.7257084212690954
  (0, 1)	1.2001028347032583
  (1, 1)	0.5007292282327223
  (2, 1)	1.3145837378651946
  (3, 1)	1.2648822869299634
  (0, 2)	1.0981924683541506
  (1, 2)	1.3145837378651946
  (2, 2)	0.38335116920405454
  (3, 2)	1.2120535167374336
  (0, 3)	1.7257084212690954
  (1, 3)	1.2648822869299634
  (2, 3)	1.2120535167374336
  (3, 3)	1.894280668672564


Matrix([
[1.22015756849392,  1.20010283470326,  1.09819246835415,  1.7257084212691],
[1.20010283470326, 0.500729228232722,  1.31458373786519, 1.26488228692996],
[1.09819246835415,  1.31458373786519, 0.383351169204055, 1.21205351673743],
[ 1.7257084212691,  1.26488228692996,  1.21205351673743, 1.89428066867256]])

In [14]:
Asp.shape

(4, 4)

In [15]:
dict(Acoo.todok().items())

NameError: name 'Acoo' is not defined

Alright, it converged! Now, what is the active power flow on the lines?

In [ ]:
network.lines_t.p0

,My line 0,My line 1,My line 2
snapshot,,,
now,66.897487,-33.333333,-33.391038


...and what are the voltage angles on the buses?

In [ ]:
network.buses_t.v_ang * 180 / np.pi

Bus,My bus 0,My bus 1,My bus 2
snapshot,,,
now,0.0,-0.875939,-0.433813


...and their mangitudes?

In [ ]:
network.buses_t.v_mag_pu

Bus,My bus 0,My bus 1,My bus 2
snapshot,,,
now,1.0,0.981199,0.99057
